<a href="https://colab.research.google.com/github/XiaoHuang0803/ColabNotebooks/blob/main/LC03_PDF_and_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pypdf
# !pip install faiss-cpu
# !pip install langchain_community
# !pip install langchain_openai
# !pip install langchain_pinecone

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("openAIAccessKey")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
pdf_path = "./ReActPaper.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents)

In [ ]:
from langchain_openai import OpenAIEmbeddings, OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
# FAISS
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_index_react")

In [ ]:
new_vectorstore = FAISS.load_local("faiss_index_react", embeddings, allow_dangerous_deserialization=True)

In [32]:
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(OpenAI(), retrieval_qa_chat_prompt)
retrieval_chain = create_retrieval_chain(new_vectorstore.as_retriever(), combine_docs_chain)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [34]:
res = retrieval_chain.invoke({"input": "Give me the gist of ReAct in 3 sentences"})
print(res["answer"])



1. ReAct is a new paradigm that combines reasoning and acting in language models for solving various tasks.
2. It uses simple prompts written in natural language to guide the model's decision making process.
3. ReAct outperforms previous approaches and provides improved interpretability, trustworthiness, and performance on a diverse set of benchmarks.
